In [ ]:
# Setup commands for installing dependencies and tools required for running a Flask web application with ngrok tunneling
!python -m spacy download en_core_web_md
!pip install flask --quiet
!pip install flask-ngrok --quiet
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2dQBXrnyOC0JCAXLXtBfGGMFGWA_2yj3NDBGtcdwqxVG9MgV1
!pip install pyngrok
!pip install flask_cors
!pip install langdetect

In [1]:
# Open the 'intents.json' file in read mode and read its contents into a variable
data_file = open('./intents.json').read()

In [2]:
import requests

# Define the URL for the translation API
url = "https://google-translation-unlimited.p.rapidapi.com/translate"

# Define the headers with required content type and RapidAPI key and host
headers = {
    "content-type": "application/x-www-form-urlencoded",
    "X-RapidAPI-Key": "c6508df8d2msh28c51d25a65f866p1c9b1ajsn7ff5b8b72944",
    "X-RapidAPI-Host": "google-translation-unlimited.p.rapidapi.com"
}


In [3]:
from langdetect import detect

# Function to translate the input text to the specified target language using the Google Translation API.
def translate_lang(text, lang):
    try:
        payload = {
          "texte": text,
          "to_lang": lang
          }
        print(payload)
        translation = requests.post(url, data=payload, headers=headers).json()['translation_data']['translation']
        print(translation)
        return translation
    except Exception as e:
        print("Error:", e)
        return None

In [4]:
# Import necessary modules
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/paws/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/paws/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/paws/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Preprocesses the input text by tokenizing, removing stopwords, and lemmatizing.
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

# Calculates the cosine similarity between two input texts after preprocessing.
def calculate_similarity(text1, text2):
    processed_text1 = preprocess_text(text1)
    processed_text2 = preprocess_text(text2)

    processed_texts = [processed_text1, processed_text2]

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_texts)

    similarity_matrix = cosine_similarity(tfidf_matrix)

    similarity_score = similarity_matrix[0][1]
    return similarity_score


In [ ]:
import json
import spacy

intents = json.loads(data_file)
nlp = spacy.load('en_core_web_md')

# Set up ngrok authentication and obtain public URL
from pyngrok import ngrok 
ngrok.set_auth_token('2dQBXrnyOC0JCAXLXtBfGGMFGWA_2yj3NDBGtcdwqxVG9MgV1')
public_url = ngrok.connect(5000).public_url
print(public_url)

# Function to determine intent based on user input
def get_intent(user_input):
    max_similarity = 0
    intent_tag = None
    for intent in intents['intents']:
        for pattern in intent['patterns']:
          if user_input!=None and pattern!=None:
            similarity = calculate_similarity(user_input.lower(), pattern.lower())
            if similarity > max_similarity:
                max_similarity = similarity
                intent_tag = intent['tag']

    return intent_tag

# Function to retrieve response based on intent tag
def get_response(intent_tag):
    for intent in intents['intents']:
        if intent['tag'] == intent_tag:
            response = intent['responses'][0]
            return response

# Function to initiate chatbot interaction
def chat(user_input):
    print("Hello! Welcome to the Wikipedia editing support chatbot.")
    while True:
        if user_input.lower() == 'exit':
            print("Goodbye! Don't hesitate to return if you need further assistance.")
            break

        detected_lang = detect(user_input)

        if detected_lang != 'en':
            translated_input = translate_lang(user_input, "en")
        else:
            translated_input = user_input

        intent_tag = get_intent(translated_input)
        response = [get_response(intent_tag),intent_tag]

        if response[0] == None:
          translated_response = "Sorry, I donot have an answer to that :("
        if detected_lang != 'en':
            translated_response = translate_lang(response[0], detected_lang)
        else:
            translated_response = response[0]

        return [translated_response,intent_tag]


# Flask application setup
from flask import Flask, request, jsonify
from flask_cors import CORS
app = Flask(__name__)
# app.secret_key = '2dQBXrnyOC0JCAXLXtBfGGMFGWA_2yj3NDBGtcdwqxVG9MgV1'
# run_with_ngrok(app)# Run ngrok with the Flask app

# Route to process user queries
CORS(app)
@app.route('/query', methods=['POST'])
def process_query():
    data = request.json
    user_query = data['query']

    # Process user query and generate response (replace with your chatbot logic)
    response = chat(user_query)

    return jsonify({'response': response})

# Run Flask app
if __name__ == '__main__':
    app.run(port = 5000)

t=2024-03-09T02:42:48+0000 lvl=warn msg="ngrok config file found at both XDG and legacy locations, using XDG location" xdg_path=/home/paws/.config/ngrok/ngrok.yml legacy_path=/home/paws/.ngrok2/ngrok.yml


https://a536-185-15-56-1.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Hello! Welcome to the Wikipedia editing support chatbot.
{'texte': 'hello', 'to_lang': 'en'}
hello
{'texte': 'Hello! Welcome to the Wikipedia editing support chatbot. How can I assist you today?', 'to_lang': 'fi'}


127.0.0.1 - - [09/Mar/2024 02:43:10] "POST /query HTTP/1.1" 200 -


...

127.0.0.1 - - [09/Mar/2024 05:13:34] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2024 05:13:49] "OPTIONS /query HTTP/1.1" 200 -


Hello! Welcome to the Wikipedia editing support chatbot.


127.0.0.1 - - [09/Mar/2024 05:13:50] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2024 05:13:56] "OPTIONS /query HTTP/1.1" 200 -


Hello! Welcome to the Wikipedia editing support chatbot.


127.0.0.1 - - [09/Mar/2024 05:13:57] "POST /query HTTP/1.1" 200 -
